logos: owkin, Pyladies Paris, Substra

# Table of contents




# Helper
(in separate notebook)
-> what you need to have installed (conda? libraries, jupyter lab)
-> few helpful funcions about jupyter lab
-> knowledge on deep learning algorithms is not required but will be useful. This said, this workshop is very new so any comments are apprepriated. Please stop me at any time if something is not clear.
-> If you wish to learn more about ML refer to: (TODO: references)


# Introduction to Federated Learning

**author**: Maria Telenczuk

TODO: few words about federated learning (FL)

### Resources

1. 1st article on FL:
    [H. Brendan McMahan and Eider Moore and Daniel Ramage and Seth Hampson and Blaise Agüera y Arcas (2017) Communication-Efficient Learning of Deep Networks from Decentralized Data, arXiv1602.05629](https://arxiv.org/abs/1602.05629)
1. [Google blogpost on FL](https://ai.googleblog.com/2017/04/federated-learning-collaborative.html)


## Quick intro to Machine Learning

**Machine learning** TODO: general idea

**ML algorithms** TODO: genaral idea

**Deep learning algorithms** TODO: general idea with some image, explain what are weights and gradients and hyperparameters.

**Data** TODO: importance of the data in ML


## Import all necessary libraries

We are using Pytorch (libary which simplifies creating deep learning algorithms)
and matplotlib (which we will use for plotting the data).
%matplotlib inline is 


In [ ]:
# get mnist dataset + understand it

import matplotlib.pyplot as plt

import os

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

## prepare the data

In [ ]:
batch_size = 64

# Create data loaders.
# n_train_data = len(training_data)
# n_val_data = int(n_train_data * 0.3)

# creat train, validate and test dataloaders
# train_alone, validate_dataset = torch.utils.data.random_split(training_data, [n_train_data - n_val_data, n_val_data])
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
# validate_dataloader = DataLoader(validate_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


#for X, y in test_dataloader:
#    print("Shape of X [N, C, H, W]: ", X.shape)
#    print("Shape of y: ", y.shape, y.dtype)

In [ ]:
fig, axes = plt.subplots(10, 10, figsize=(8, 8))
fig.subplots_adjust(hspace=0.1, wspace=0.1)

for i, ax in enumerate(axes.flat):
    image, label = training_data[i]
    ax.imshow(image.squeeze(), cmap="gray", interpolation='nearest')
    ax.text(0.05, 0.05, str(label),
            transform=ax.transAxes, color='white')
    ax.set_xticks([])
    ax.set_yticks([])


In [ ]:
classes = {
    0: 't-shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

## Prepare algorithm

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, lambda_dropout=0.0):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        n_hidden = 32
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.Dropout(lambda_dropout),
            nn.ReLU(),
            nn.Linear(n_hidden, 10),
            # nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## Train the model

In [ ]:
model = NeuralNetwork()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

n_epochs = 5
for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        image, label = data
        if image.shape[0] < 64:
            continue

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % batch_size == batch_size-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

In [ ]:
def predict_model(model, test_dataloader):
    model.eval()
    prediction = []
    with torch.no_grad():
        for image, label in test_dataloader:
            # image, label = X.to(self._device), y.to(self._device)
            pred = model(image)

            # to view predicted labels uncomment following lines:
            # print('predicted:', pred.argmax(1))
            # print('true:',label)
            prediction.append(pred)
    return prediction

prediction = predict_model(model, test_dataloader)
        

## calculate the score given the data and the predictions

In [ ]:
def get_score(dataloader, next_pred):
    loss_fn = nn.CrossEntropyLoss()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    for [X, y], y_pred in zip(dataloader, next_pred):
        X, y = X.to(device), y.to(device)
        test_loss += loss_fn(y_pred, y).item()
        correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    
    accuracy = (100*correct)
    avg_loss = test_loss
    return accuracy, avg_loss

accuracy, loss = get_score(test_dataloader, prediction)

print('accuracy is:', accuracy)
print('loss is:', loss)

You can predict the output based on this data

Now, we will define a simple pytorch model to train this data

In [ ]:
'''
import torch
from torch import nn, optim

input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)
'''

# Introduction to ML
- data, mnistfashion or other
- simple model
- privacy. give examples: (personal data, phone/e-mail data, healthcare data etc)
# show an image on classical ML
# privacy issues
# explain the concept of FL
# FL challenges (performance and privacy, amount of data, trust of the algorithm if not seen, trust of the data of others etc
This very much differs on the use case. The first article (give reference, google) describing the concept of FL was provided by Google for their usecase. A lot of phone users write their messages and it would be great to be able to suggest to them the rest of the message. It can be very personal what you write so the suggestion should be personalized, but you most likely won't provide enough data to train the deep learnig algorithm. On the other side it would be a security breach if all the messages, from all the phones were send to the single server for training. Google proposed that the model will be trained on each phone for one or more epochs and then the model, not the data will be send to the centralized location. This could be for example weights. In the centralized location the server then combines those weights, for example averages them and sends back to each of the phones. Those updated weights they then use for further training.

In Owkin the use case is very different. The clients or sites are only very few but they have much larger data. The data is usually very sensitive and no information can leave the site. Nobody should even know how much of the data is on which site. You won't be able to check if it is well preprocessed or for the missing data so as you imagine it comes with additional challenges and I suppose, in all cases, some trust.

Another challenge is a performance. The bottleneck of FL is communication: sending the model back and fourth and waiting until the processing is finished. Especially if the model is very big, and even more so if there are a lot of sites.




# so let's now imagine...

In [ ]:
# We divide the data to 3 sites
n_train_data = len(training_data)
n_site1, n_site2 = int(n_train_data * 0.5), int(n_train_data * 0.2)
n_site3 = n_train_data - n_site1 - n_site2
site1, site2, site3 = torch.utils.data.random_split(training_data, [n_site1, n_site2, n_site3])
site1_dataloader = DataLoader(site1, batch_size=batch_size, shuffle=True)
site2_dataloader = DataLoader(site2, batch_size=batch_size, shuffle=True)
site3_dataloader = DataLoader(site3, batch_size=batch_size, shuffle=True)
print(len(site1))

## update our model to fl settings

In [ ]:
import copy

model = NeuralNetwork()
learning_rate = 0.1
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

def train(model, dataloader):
    # note: this is very simplified and won't work with every optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        num_batches = len(dataloader)
        # get the inputs; data is a list of [inputs, labels]
        image, label = data
        if image.shape[0] < 64:
            continue

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % batch_size == batch_size-1:
            #print(loss.item())
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / batch_size))
            running_loss = 0.0
    return model


def avg_weights(weight1, weight2, weight3):
    
    mean_weights = []
    for w1, w2, w3 in zip(weight1, weight2, weight3):
        next_weight = (w1 + w2 + w3) / 3
        #next_weight = torch.mean(w1, w2, w3)
        # loop through all the weights
        #for i in range(1, len(weights)):
        #    next_weight += weight
        #next_weigth /= idx
        mean_weights.append(next_weight)
    return mean_weights
    

def get_weights(model):
    return model.parameters()

def set_weights(model, new_weights):
    with torch.no_grad():
        for param, new_param in zip(model.parameters(), new_weights):
            param.data[:] = new_param.data
    return model
    
n_epochs = 5
for epoch in range(n_epochs):  # loop over the dataset multiple times
    
    # train on site1
    # beta = 0.5 #The interpolation parameter
    '''
    params1 = model1.named_parameters()
    params2 = model2.named_parameters()

    dict_params2 = dict(params2)

    for name1, param1 in params1:
        if name1 in dict_params2:
            dict_params2[name1].data.copy_(beta*param1.data + (1-beta)*dict_params2[name1].data)

     model.load_state_dict(dict_params2)
    ''' 
    
    model1 = train(copy.deepcopy(model), site1_dataloader)
    model2 = train(copy.deepcopy(model), site2_dataloader)
    model3 = train(copy.deepcopy(model), site3_dataloader)

    params_m1 = get_weights(model1)
    params_m2 = get_weights(model2)
    params_m3 = get_weights(model3)
    
    mean_weights = avg_weights(params_m1, params_m2, params_m3)
    
    model = set_weights(model, mean_weights)
    print('continue to epoch, ', epoch)
    
print('Finished Training')

In [ ]:
# predict
prediction = predict_model(model, test_dataloader)
accuracy, loss = get_score(test_dataloader, prediction)

print('accuracy is:', accuracy)
print('loss is:', loss)

## Security

## hacking federated learning
If we just left the model as it is it would have been easy to hack on the model.

Now we will use slightly different data for site1 and site2

In [ ]:
import torchvision as torchvision
import torch.utils.data.sampler as sampler
def get_indices(dataset, class_name):
    indices =  []
    for i in range(len(dataset.dataset.targets)):
        if dataset.dataset.targets[i] in class_name:
            indices.append(i)
    return indices


#dataset = torchvision.datasets.CIFAR10(root='./data',
#                           transform=torchvision.transforms.ToTensor())

idx = get_indices(site1, [0, 1, 2])
site1_newloader = DataLoader(training_data, batch_size=64, sampler=sampler.SubsetRandomSampler(idx))

# idx = get_indices(site2, [5, 6])
# site2_newloader = DataLoader(site2, batch_size=64, sampler=sampler.SubsetRandomSampler(idx))

for idx, (data, target) in enumerate(site1_newloader):
    pass
print(idx, target)
# for idx, (data, target) in enumerate(site2_newloader):
#    print(idx, target)

In [ ]:
model1 = train(copy.deepcopy(model), site1_newloader)


In [ ]:
#image, label = len(training_data)
idx = 0

*_, last_m1 = model1.parameters() # for a better understanding check PEP 448
*_, last_m = model.parameters() 
# print(last)


for idx, weight in enumerate(last_m1 - last_m):
    # m = m1.data - m2.data
    #if idx == 5:
    #    print(m1, m2)
    #    print(m)
    if weight > 0:
        print(idx, ':', classes[idx])
    #if i > 3:
    #    break
    # print(v.detach().numpy().shape)
    #plt.imshow(v.detach().numpy())
    # to view predicted labels uncomment following lines:
    # print('predicted:', pred.argmax(1))
    # print('true:',label)
    #if(v != 0):
    #     print(vocab[i])
    idx += 1

So in fact we don't want the 

Explain secure aggregation

- adding noise (more secure but could harm performance)
- so aggregate in a way that nobody can view the weights of others
- so how could we do that? use homomorphic encryption



## What is homomorphic encryption

homomorphic encryption: gives possibility to perform some computations on encrypted values without decrypting them

First, a public key lets you encrypt numbers. A private key lets you decrypt
encrypted numbers. An encrypted value is called a ciphertext, and an unencrypted value
is called a plaintext. 

phe library? or something used in Substra?
give some resources

Now, let’s return to the problem of secure aggregation. Given your new knowledge that
you can add together numbers you can’t see, the answer becomes plain. The person who
initializes the model sends a public_key to Bob, Alice, and Sue so they can each encrypt
their weight updates

Then, Bob, Alice, and Sue (who don’t have the private key) talk directly
to each other and accumulate all their gradients into a single, final update that’s sent back to
the model owner, who decrypts it with the private_key.

In substra: Encryption function using python's pre-installed packages HMAC and hashlib.
    We are using SHA256 (it is equal in security to SHA512)
    scaffold (??)

In [ ]:
'''
model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0

# note that in production the n_length should be at least 1024
public_key, private_key = phe.generate_paillier_keypair(n_length=128)

def train_and_encrypt(model, input, target, pubkey):
    new_model = train(copy.deepcopy(model), input, target, iterations=1)

    encrypted_weights = list()
    for val in new_model.weight.data[:,0]:
        encrypted_weights.append(public_key.encrypt(val))
    ew = np.array(encrypted_weights).reshape(new_model.weight.data.shape)
    
    return ew
'''

In [ ]:
'''
# enable encrypting

import syft.frameworks.tenseal as ts

# hook PyTorch to add extra functionalities like the ability to encrypt torch tensors
hook = sy.TorchHook(th)
'''

In [ ]:
import phe

import syft as sy

# hook PyTorch to add extra functionalities like the ability to encrypt torch tensors
hook = sy.TorchHook(torch)

model = NeuralNetwork()
learning_rate = 0.1
criterion = nn.CrossEntropyLoss()

# this should be more secure for production
public_key, private_key = phe.generate_paillier_keypair(n_length=128)

def train_and_encrypt(model, dataloader, public_key):
    # note: this is very simplified and won't work with every optimizer
    
    new_model = train(copy.deepcopy(model), dataloader)
    
    encrypted_layers = list()
    for val in new_model.parameters():
        encrypted_weights = list()
        # matrix_encrypted = val.data.encrypt("ckks", public_key=public_keys)
        # data = val.data[:,0]
        # print(val.data[:,0])
        # print('vs', val.data)
        for num in val:
            print(num)
            encrypted_weights.append(public_key.encrypt(num))
        enc_weights = np.array(encrypted_layers).reshape(new_model.weight.data.shape)
        encrypted_layers.append(encrypted_weights)
    
    return enc_weights


def avg_weights(weight1, weight2, weight3):
    
    mean_weights = []
    weight1 + weight2 + weight3
    for w1, w2, w3 in zip(weight1, weight2, weight3):
        next_weight = (w1 + w2 + w3) / 3
        mean_weights.append(next_weight)
    return mean_weights
    

def get_weights(model):
    return model.parameters()

def set_weights(model, new_weights):
    with torch.no_grad():
        for param, new_param in zip(model.parameters(), new_weights):
            param.data[:] = new_param.data
    return model
    
n_epochs = 5
for epoch in range(n_epochs):  # loop over the dataset multiple times
    
    w1 = train_and_encrypt(copy.deepcopy(model), site1_dataloader, public_key)
    w2 = train_and_encrypt(copy.deepcopy(model), site2_dataloader, public_key)
    w3 = train_and_encrypt(copy.deepcopy(model), site3_dataloader, public_key)

    
    mean_weights = avg_weights(w1, w2, w3)
    
    model = set_weights(model, mean_weights)
    print('continue to epoch, ', epoch)
    
print('Finished Training')

other concepts:  federated learning,
homomorphic encryption, differential privacy, and secure multi-party computation are all
built in 

Two words about Substra and Connect

different strategies

two words about Connectlib

# Resources

